In [1]:
import time
import math

import numpy as np
import matplotlib.pyplot as plt
import skimage as ski
import skimage.io
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import nn
import layers


Instructions for updating:
Use the retry module or similar alternatives.


# Task 3 - Tensorflow MNIST

Define and train a tensorflow model which is equivalent to the regularized model from Task 2. Define an identical architecture and training parameters in order to reproduce the results. Use the convolution operations from `tf.nn.conv2d` or `tf.contrib.layers.convolution2d`. Study the official documentation for the [convolution](https://www.tensorflow.org/versions/master/api_docs/python/nn.html#convolution) suport in Tensorflow. Visualize the trained filters from the first layer during training, as in Task 2.

An example of using convolutions defined in the `tf.contrib` package is shown below. If you prefer to use `tf.nn.conv2d`, please consult the official [tutorial](https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#build-a-multilayer-convolutional-network).

In [ ]:
import tensorflow.contrib.layers as layers

def build_model(inputs, labels, num_classes):
    weight_decay = ...
    conv1sz = ...
    fc3sz = ...
    with tf.contrib.framework.arg_scope([layers.convolution2d],
            kernel_size=5, stride=1, padding='SAME', activation_fn=tf.nn.relu,
            weights_initializer=layers.variance_scaling_initializer(),
            weights_regularizer=layers.l2_regularizer(weight_decay)):

        net = layers.convolution2d(inputs, conv1sz, scope='conv1')
        # ostatak konvolucijskih i pooling slojeva
        ...

    with tf.contrib.framework.arg_scope([layers.fully_connected],
            activation_fn=tf.nn.relu,
            weights_initializer=layers.variance_scaling_initializer(),
            weights_regularizer=layers.l2_regularizer(weight_decay)):

        # sada definiramo potpuno povezane slojeve
        # ali najprije prebacimo 4D tenzor u matricu
        net = layers.flatten(inputs)
        net = layers.fully_connected(net, fc3sz, scope='fc3')

    logits = layers.fully_connected(net, num_classes, activation_fn=None, scope='logits')
    loss = ...

    return logits, loss


In [2]:
DATA_DIR = 'MNIST original'
SAVE_DIR = 'task3_out'
import os
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

config = {}
config['max_epochs'] = 8
config['batch_size'] = 50
config['save_dir'] = SAVE_DIR
config['weight_decay'] = 1e-3
config['lr_policy'] = {1:{'lr':1e-1}, 3:{'lr':1e-2}, 5:{'lr':1e-3}, 7:{'lr':1e-4}}


In [3]:
#np.random.seed(100) 
np.random.seed(int(time.time() * 1e6) % 2**31)
dataset = input_data.read_data_sets(DATA_DIR, one_hot=True)
train_x = dataset.train.images
train_x = train_x.reshape([-1, 28, 28, 1])
train_y = dataset.train.labels

valid_x = dataset.validation.images
valid_x = valid_x.reshape([-1, 28, 28, 1])
valid_y = dataset.validation.labels

test_x = dataset.test.images
test_x = test_x.reshape([-1, 28, 28, 1])
test_y = dataset.test.labels

train_mean = train_x.mean()
train_x -= train_mean
valid_x -= train_mean
test_x -= train_mean

weight_decay = config['weight_decay']


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST original\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST original\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST original\t10k-images-idx3-ubyte.gz
Extracting MNIST original\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [233]:
# https://stackoverflow.com/questions/42286426/what-is-the-difference-between-xavier-initializer-and-xavier-initializer-conv2d
# itializer is designed to keep the scale of the gradients roughly the same in all layers
from tensorflow.contrib.layers import xavier_initializer_conv2d
from tensorflow.contrib.layers import xavier_initializer


##
# Dimensions of the tensor is: [batch, height, width, channels]
##


def Convolution(_input, w, b, n_strides=1):
    # strides determines how much the window shifts by in each of the dimensions.
    # The typical use sets the first (the batch) and last (the depth) stride to 1.
    node = tf.nn.conv2d(_input, w, strides=[1, n_strides, n_strides, 1], padding='SAME')
    node = tf.nn.bias_add(node, b)
    return node


def ReLU(_input):
    return tf.nn.relu(_input)


def MaxPooling(_input, ks=2, n_strides=2):
    # https://stackoverflow.com/questions/38601452/the-usages-of-ksize-in-tf-nn-max-pool
    # ksize - kernel size. eg. [1,2,2,1] - kernel 2x2
    
    # strides - determines how much the window shifts by in each of the dimensions.
    # The typical use sets the first (the batch) and last (the depth) stride to 1.
    
    # https://stackoverflow.com/questions/34642595/tensorflow-strides-argument
    # The input to the convolution has shape=[1, 32, 32, 1].
    # If you specify strides=[1,1,1,1] with padding=SAME, then the output of the filter will be [1, 32, 32, 8].
    return tf.nn.max_pool(_input, ksize=[1, ks, ks, 1], strides=[1, n_strides, n_strides, 1], padding='SAME')


def Flatten(_input, w):
    # Given tensor, this operation returns a tensor that has the same values as tensor with shape shape.
    # this is needed tbecause fully connected layer is simple "1-d vector"
    return tf.reshape(_input, [-1, w.shape[0]])


def FC(_input, w, b):
    return tf.matmul(_input, w) + b


def SoftmaxCrossEntropyWithLogits(logits, labels):
    # softmax_cross_entropy_with_logits_v2 is new version of deprecated softmax_cross_entropy_with_logits
    # https://stats.stackexchange.com/questions/327348/how-is-softmax-cross-entropy-with-logits-different-from-softmax-cross-entropy-wi
    # "in supervised learning one doesn't need to backpropagate to labels"
    # thats why tf.stop_gradient in softmax_cross_entropy_with_logits_v2
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf.stop_gradient(labels)))


def RegularizedLoss(data_loss, regularizers):
    return data_loss + weight_decay*regularizers


def L2Regularizer(weights):
    return tf.nn.l2_loss(weights)
    

In [234]:
n_classes = dataset.train.labels.shape[1] # [0] - num of labels, [1] - num of classes


tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Yoh_ = tf.placeholder(tf.float32, [None, n_classes])

net=X
regularizers=0
#net = tf.reshape(X, shape=[-1, 28, 28, 1])

w_conv1 = tf.get_variable('w_conv1', [5, 5, 1, 16], initializer=xavier_initializer_conv2d())
b_conv1 = tf.Variable(tf.zeros([16]), name='b_conv1')
net = Convolution(net, w_conv1, b_conv1)
regularizers += L2Regularizer(w_conv1)
net = MaxPooling(net)
net = ReLU(net)

w_conv2 = tf.get_variable('w_conv2', [5, 5, 16, 32], initializer=xavier_initializer_conv2d())
b_conv2 = tf.Variable(tf.zeros([32]), name='b_conv2')
net = Convolution(net, w_conv2, b_conv2)
regularizers += L2Regularizer(w_conv2)
net = MaxPooling(net)
net = ReLU(net)

## 7x7
w_fc3 = tf.get_variable('w_fc3', [7*7*32, 512], initializer=xavier_initializer())
b_fc3 = tf.Variable(tf.zeros([512]), name='b_fc3')
net = Flatten(net, w_fc3)
net = FC(net, w_fc3, b_fc3)
regularizers += L2Regularizer(w_fc3)
net = ReLU(net)

w_fc4 = tf.get_variable('w_fc4', [512, n_classes], initializer=xavier_initializer())
b_fc4 = tf.Variable(tf.zeros([n_classes]), name='b_fc4')
net = FC(net, w_fc4, b_fc4)

data_loss = SoftmaxCrossEntropyWithLogits(net, Yoh_)
loss = RegularizedLoss(data_loss, regularizers)

lr = tf.placeholder(tf.float32)
train_step =  tf.train.GradientDescentOptimizer(lr).minimize(loss)


In [241]:
def train(session, train_x, train_y, valid_x, valid_y, config, logits):
    session.run(tf.initialize_all_variables())

    lr_policy = config['lr_policy']
    batch_size = config['batch_size']
    max_epochs = config['max_epochs']
    save_dir = config['save_dir']
    num_examples = train_x.shape[0]
    assert num_examples % batch_size == 0
    num_batches = num_examples // batch_size

    for epoch in range(1, max_epochs+1):
        if epoch in lr_policy:
            solver_config = lr_policy[epoch]
        cnt_correct = 0

        permutation_idx = np.random.permutation(num_examples)
        train_x = train_x[permutation_idx]
        train_y = train_y[permutation_idx]

        for i in range(num_batches):
            # store mini-batch to ndarray
            batch_x = train_x[i*batch_size:(i+1)*batch_size, :]
            batch_y = train_y[i*batch_size:(i+1)*batch_size, :]

            data_dict = {X: batch_x, Yoh_: batch_y, lr:solver_config['lr']}
            ##logits = forward_pass(net, batch_x)
            #logits_val = session.run(logits, feed_dict=data_dict)
            ##loss_val = loss.forward(logits, batch_y)
            #loss_val = session.run(loss, feed_dict=data_dict)
            #session.run(train_step, feed_dict=data_dict)
            # optimization - one execution of session.run
            logits_val, loss_val, _ = session.run([logits, loss, train_step], feed_dict=data_dict)

            # compute classification accuracy
            yp = np.argmax(logits_val, 1)
            yt = np.argmax(batch_y, 1)
            cnt_correct += (yp == yt).sum()
            ##grads = backward_pass(net, loss, logits, batch_y)
            ##sgd_update_params(grads, solver_config)

            if i % 5 == 0:
                print("epoch %d, step %d/%d, batch loss = %.2f" % (epoch, i*batch_size, num_examples, loss_val))
            if i % 100 == 0:
                w = session.run(w_conv1)
                draw_conv_filters(epoch, i*batch_size, "conv1", w, save_dir)
            if i > 0 and i % 50 == 0:
                print("Train accuracy = %.2f" % (cnt_correct / ((i+1)*batch_size) * 100))
                
        print("Train accuracy = %.2f" % (cnt_correct / num_examples * 100))
        evaluate(session, "Validation", valid_x, valid_y, config, logits)


In [238]:
def evaluate(session, name, x, y, config, logits):
    print("\nRunning evaluation: ", name)
    batch_size = config['batch_size']
    num_examples = x.shape[0]
    assert num_examples % batch_size == 0
    num_batches = num_examples // batch_size
    cnt_correct = 0
    loss_avg = 0

    for i in range(num_batches):
        batch_x = x[i*batch_size:(i+1)*batch_size, :]
        batch_y = y[i*batch_size:(i+1)*batch_size, :]
        
        ##logits = forward_pass(net, batch_x)
        data_dict = {X: batch_x, Yoh_: batch_y}
        logits_val = session.run(logits, feed_dict=data_dict)
        
        yp = np.argmax(logits_val, 1)
        yt = np.argmax(batch_y, 1)
        cnt_correct += (yp == yt).sum()
        loss_val = session.run(loss, feed_dict=data_dict)
        loss_avg += loss_val
        #print("step %d / %d, loss = %.2f" % (i*batch_size, num_examples, loss_val / batch_size))
    valid_acc = cnt_correct / num_examples * 100
    loss_avg /= num_batches
    print(name + " accuracy = %.2f" % valid_acc)
    print(name + " avg loss = %.2f\n" % loss_avg)


In [239]:
def draw_conv_filters(epoch, step, name, weights, save_dir):
    k, k, C, num_filters = weights.shape

    w = weights.copy().swapaxes(0, 3).swapaxes(1,2)
    w = w.reshape(num_filters, C, k, k)
    w -= w.min()
    w /= w.max()

    border = 1
    cols = 8
    rows = math.ceil(num_filters / cols)
    width = cols * k + (cols-1) * border
    height = rows * k + (rows-1) * border
    #for i in range(C):
    for i in range(1):
        img = np.zeros([height, width])
        for j in range(num_filters):
            r = int(j / cols) * (k + border)
            c = int(j % cols) * (k + border)
            img[r:r+k,c:c+k] = w[j,i]
        filename = '%s_epoch_%02d_step_%06d_input_%03d.png' % (name, epoch, step, i)
        ski.io.imsave(os.path.join(save_dir, filename), img)


In [240]:
session = tf.Session()
train(session, train_x, train_y, valid_x, valid_y, config, net)
evaluate(session, "Test", test_x, test_y, config, net)

epoch 1, step 0/55000, batch loss = 2.70


C:\dev\software\anaconda3_5.0.0_3.6_x64\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


epoch 1, step 250/55000, batch loss = 2.59
epoch 1, step 500/55000, batch loss = 2.45
epoch 1, step 750/55000, batch loss = 2.28
epoch 1, step 1000/55000, batch loss = 2.41
epoch 1, step 1250/55000, batch loss = 1.49
epoch 1, step 1500/55000, batch loss = 1.12
epoch 1, step 1750/55000, batch loss = 1.45
epoch 1, step 2000/55000, batch loss = 0.84
epoch 1, step 2250/55000, batch loss = 1.00
epoch 1, step 2500/55000, batch loss = 0.81
Train accuracy = 55.80
epoch 1, step 2750/55000, batch loss = 0.66
epoch 1, step 3000/55000, batch loss = 0.71
epoch 1, step 3250/55000, batch loss = 0.82
epoch 1, step 3500/55000, batch loss = 0.56
epoch 1, step 3750/55000, batch loss = 0.80
epoch 1, step 4000/55000, batch loss = 0.77
epoch 1, step 4250/55000, batch loss = 0.74
epoch 1, step 4500/55000, batch loss = 0.65
epoch 1, step 4750/55000, batch loss = 0.90
epoch 1, step 5000/55000, batch loss = 0.64
Train accuracy = 71.88
epoch 1, step 5250/55000, batch loss = 0.66
epoch 1, step 5500/55000, batch l

KeyboardInterrupt: 

# Task 4: Tensorflow CIFAR 10

[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset contains 50000 images for training and validation, and 10000 test images. The images have dimensions 32x32 and they belong to 10 classes. Download the dataset version for Python [here](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz). Use the following code to load and prepair the dataset.

In [ ]:
import os
import pickle
import numpy as np

def shuffle_data(data_x, data_y):
    indices = np.arange(data_x.shape[0])
    np.random.shuffle(indices)
    shuffled_data_x = np.ascontiguousarray(data_x[indices])
    shuffled_data_y = np.ascontiguousarray(data_y[indices])
    return shuffled_data_x, shuffled_data_y

def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

DATA_DIR = '/path/to/data/'


train_x = np.ndarray((0, img_height * img_width * num_channels), dtype=np.float32)
train_y = []
for i in range(1, 6):
    subset = unpickle(os.path.join(DATA_DIR, 'data_batch_%d' % i))
    train_x = np.vstack((train_x, subset['data']))
    train_y += subset['labels']
train_x = train_x.reshape((-1, num_channels, img_height, img_width)).transpose(0,2,3,1)
train_y = np.array(train_y, dtype=np.int32)

subset = unpickle(os.path.join(DATA_DIR, 'test_batch'))
test_x = subset['data'].reshape((-1, num_channels, img_height, img_width)).transpose(0,2,3,1).astype(np.float32)
test_y = np.array(subset['labels'], dtype=np.int32)

valid_size = 5000
train_x, train_y = shuffle_data(train_x, train_y)
valid_x = train_x[:valid_size, ...]
valid_y = train_y[:valid_size, ...]
train_x = train_x[valid_size:, ...]
train_y = train_y[valid_size:, ...]
data_mean = train_x.mean((0,1,2))
data_std = train_x.std((0,1,2))

train_x = (train_x - data_mean) / data_std
valid_x = (valid_x - data_mean) / data_std
test_x = (test_x - data_mean) / data_std



Your task is to train a convolutional model in Tensorflow. We propose a simple model which should yield about 70% accuracy in image classification.

In [ ]:
conv(16,5) -> relu() -> pool(3,2) -> conv(32,5) -> relu() -> pool(3,2) -> fc(256) -> relu() -> fc(128) -> relu() -> fc(10)

Here `conv(16,5)` represents a convolution with 16 feature maps and filter dimensions 5x5, `pool(3,2)` is a max-pooling layer operating on patches 3x3 and the stride 2.

Write the function `evaluate` which compares the predicted and correct class indices and determines the following classification performance indicators: overall classification accuracy, confusion matrix, as well as precision and recall for particular classes. In the implementation, first determine the confusion matrix, and then use it to determine all other indicators. During training, invoke `evaluate` after each epoch both on the training and on the validation dataset, and graph the average loss, the training rate and overall classification accuracy. We recommend that function receives the data, the correct class indices, and the required tensorflow operations. Be careful not to invoke the training operation. The function should output the recovered indicators to the console.

> A typical loss graph when the training proceeds well.

Visualize random initializations and the trained filters from the first layer. You can access the variable which holds the weight of the first layer by invoking the `tf.contrib.framework.get_variables` method with the scope in which the variable is used in the model. We supply an example of how that might look like below. The scope will depend on the code which you actually used while defining the graph.

In [ ]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

conv1_var = tf.contrib.framework.get_variables('model/conv1_1/weights:0')[0]
conv1_weights = conv1_var.eval(session=sess)
draw_conv_filters(0, 0, conv1_weights, SAVE_DIR)


We also provide code which you can use for visualization:

In [ ]:
def draw_conv_filters(epoch, step, weights, save_dir):
    w = weights.copy()
    num_filters = w.shape[3]
    num_channels = w.shape[2]
    k = w.shape[0]
    assert w.shape[0] == w.shape[1]
    w = w.reshape(k, k, num_channels, num_filters)
    w -= w.min()
    w /= w.max()
    border = 1
    cols = 8
    rows = math.ceil(num_filters / cols)
    width = cols * k + (cols-1) * border
    height = rows * k + (rows-1) * border
    img = np.zeros([height, width, num_channels])
    for i in range(num_filters):
        r = int(i / cols) * (k + border)
        c = int(i % cols) * (k + border)
        img[r:r+k,c:c+k,:] = w[:,:,:,i]
    filename = 'epoch_%02d_step_%06d.png' % (epoch, step)
    ski.io.imsave(os.path.join(save_dir, filename), img)



> CIFAR-10: random initializations (top) and the learned filters in the first layer (bottom) with regularization lambda = 0.0001.
    

Visualize 20 incorrectly classified images with the largest loss and output the correct class and the top 3 predicted classes. Pay attention that in order to visualize image, you first need to undo the normalization of the mean value and variance:

In [ ]:
import skimage as ski
import skimage.io

def draw_image(img, mean, std):
    img *= std
    img += mean
    img = img.astype(np.uint8)
    ski.io.imshow(img)
    ski.io.show()



We provide the code for producing graphs below:

In [ ]:
def plot_training_progress(save_dir, data):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,8))

    linewidth = 2
    legend_size = 10
    train_color = 'm'
    val_color = 'c'

    num_points = len(data['train_loss'])
    x_data = np.linspace(1, num_points, num_points)
    ax1.set_title('Cross-entropy loss')
    ax1.plot(x_data, data['train_loss'], marker='o', color=train_color,
                     linewidth=linewidth, linestyle='-', label='train')
    ax1.plot(x_data, data['valid_loss'], marker='o', color=val_color,
                     linewidth=linewidth, linestyle='-', label='validation')
    ax1.legend(loc='upper right', fontsize=legend_size)
    ax2.set_title('Average class accuracy')
    ax2.plot(x_data, data['train_acc'], marker='o', color=train_color,
                     linewidth=linewidth, linestyle='-', label='train')
    ax2.plot(x_data, data['valid_acc'], marker='o', color=val_color,
                     linewidth=linewidth, linestyle='-', label='validation')
    ax2.legend(loc='upper left', fontsize=legend_size)
    ax3.set_title('Learning rate')
    ax3.plot(x_data, data['lr'], marker='o', color=train_color,
                     linewidth=linewidth, linestyle='-', label='learning_rate')
    ax3.legend(loc='upper left', fontsize=legend_size)

    save_path = os.path.join(save_dir, 'training_plot.pdf')
    print('Plotting in: ', save_path)
    plt.savefig(save_path)



In [ ]:
plot_data = {}
plot_data['train_loss'] = []
plot_data['valid_loss'] = []
plot_data['train_acc'] = []
plot_data['valid_acc'] = []
plot_data['lr'] = []
for epoch_num in range(1, num_epochs + 1):
    train_x, train_y = shuffle_data(train_x, train_y)
    for step in range(num_batches):
        offset = step * batch_size 
        # s ovim kodom pazite da je broj primjera djeljiv s batch_size
        batch_x = train_x[offset:(offset + batch_size), ...]
        batch_y = train_y[offset:(offset + batch_size)]
        feed_dict = {node_x: batch_x, node_y: batch_y}
        start_time = time.time()
        run_ops = [train_op, loss, logits]
        ret_val = sess.run(run_ops, feed_dict=feed_dict)
        _, loss_val, logits_val = ret_val
        duration = time.time() - start_time
        if (step+1) % 50 == 0:
            sec_per_batch = float(duration)
            format_str = 'epoch %d, step %d / %d, loss = %.2f (%.3f sec/batch)'
            print(format_str % (epoch_num, step+1, num_batches, loss_val, sec_per_batch))

    print('Train error:')
    train_loss, train_acc = evaluate(logits, loss, train_x, train_y)
    print('Validation error:')
    valid_loss, valid_acc = evaluate(logits, loss, valid_x, valid_y)
    plot_data['train_loss'] += [train_loss]
    plot_data['valid_loss'] += [valid_loss]
    plot_data['train_acc'] += [train_acc]
    plot_data['valid_acc'] += [valid_acc]
    plot_data['lr'] += [lr.eval(session=sess)]
    plot_training_progress(SAVE_DIR, plot_data)



If you have access to a GPU, you might want to try obtaining better results with a more powerful model. In that case, [here](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130) you can find a review of state-of-the-art results on this dataset. As you see, best approaches achieve around 96% overall classification accuracy. Two important tricks to achieve this are image upsampling and jittering. Image upsampling ensures that early convolutions detect very low level features, while jittering prevents overfitting. Without these techniques, it will be very hard for you to achieve more than 90% overall classification accuracy.